In [72]:
from database import *

def ticks_to_kbars(ticks, interval='5Min'):

    kbars = pd.DataFrame()
    
    kbars['open'] = ticks['close'].resample(interval).first()
    kbars['close'] = ticks['close'].resample(interval).last()
    kbars['high'] = ticks['close'].resample(interval).max()
    kbars['low'] = ticks['close'].resample(interval).min()
    kbars['volume'] = ticks['volume'].resample(interval).sum()
    
    kbars.dropna(inplace=True)
    
    return kbars

In [73]:
import talib

def get_technical_indicator(kbars):
    
#     kbars['rsi'] = talib.RSI(kbars.close, timeperiod=14)
    kbars['cci'] = talib.CCI(kbars.high, kbars.low, kbars.close, timeperiod=14)
    
    macd, macdsignal, macdhist = talib.MACD(kbars.close, fastperiod=5, slowperiod=34, signalperiod=5)
    
    kbars['macd'] = macd
    kbars['macdsignal'] = macdsignal
    kbars['macdhist'] = macdhist
    
    return kbars

In [74]:
api = sj.Shioaji()
api.login(
    person_id="",
    passwd="",
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done."))

# api.logout()

connection = sqlite3.connect('data.db')

Exception ignored in: <function Shioaji.__del__ at 0x7faa345d3310>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/shioaji/shioaji.py", line 652, in __del__
    self._solace.__del__()
  File "shioaji/backend/solace/api.pyx", line 1232, in shioaji.backend.solace.api.SolaceAPI.__del__
  File "shioaji/backend/solace/api.pyx", line 327, in shioaji.backend.solace.api.SolaceAPI.logout
  File "/opt/anaconda3/lib/python3.8/site-packages/shioaji/utils.py", line 49, in raise_resp_error
    raise Exception(resp)
Exception: {}


Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done.
<SecurityType.Future: 'FUT'> fetch done.
<SecurityType.Stock: 'STK'> fetch done.
<SecurityType.Option: 'OPT'> fetch done.


In [98]:
code = '4551'

tw_calendar = get_calendar('XTAI')

date = pd.to_datetime('2020-3-31')
prev_trading_date = tw_calendar.previous_close(date).date()

ticks = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
kbars = ticks_to_kbars(ticks)
kbars = get_technical_indicator(kbars)
kbars = kbars[pd.to_datetime(prev_trading_date).replace(hour=13, minute=30):]

kbars

,open,close,high,low,volume,cci,macd,macdsignal,macdhist
ts,,,,,,,,,
2020-03-30 13:30:00,114.0,114.0,114.0,114.0,74,-110.774411,1.146300,1.396949,-0.250649
2020-03-30 14:30:00,114.0,114.0,114.0,114.0,3,-93.333333,0.952273,1.248723,-0.296451
2020-03-31 09:00:00,111.5,114.0,115.0,111.5,280,-156.989247,0.812174,1.103207,-0.291033
2020-03-31 09:05:00,114.0,112.0,114.0,112.0,95,-219.806763,0.156261,0.787558,-0.631297
2020-03-31 09:10:00,112.0,111.0,112.0,109.0,226,-287.259259,-0.535194,0.346641,-0.881835
2020-03-31 09:15:00,111.0,109.5,111.0,109.5,81,-219.062181,-1.373914,-0.226877,-1.147037
2020-03-31 09:20:00,109.5,111.5,111.5,109.5,88,-127.004219,-1.322559,-0.592105,-0.730454
2020-03-31 09:25:00,111.5,110.5,112.0,110.5,37,-94.824962,-1.541278,-0.908496,-0.632782
2020-03-31 09:30:00,110.5,111.5,111.5,110.5,24,-72.980910,-1.373210,-1.063400,-0.309809


In [5]:
def day_trading_backtest(code, date, connection, api):
    
    tw_calendar = get_calendar('XTAI')
    prev_trading_date = tw_calendar.previous_close(date).date()
    
    ticks = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
    kbars = ticks_to_kbars(ticks)
    kbars = get_technical_indicator(kbars)
    kbars = kbars[date:]
            
    entry_price = 0
    entry_time = None
    
    exit_price = 0
    exit_time = None
    
    position = 0
            
    for ts in range(len(kbars)):
        
        current_time = kbars.iloc[ts].name + pd.Timedelta(minutes=5)
        print(' start current_time = {}'.format(current_time))
        current_price = kbars['close'][ts]
        print('current_price = {}'.format(current_price))
#         macdhist = kbars.iloc[ts]['macdhist']
#         print('macdhist = {}'.format(macdhist))
#         cci = kbars.iloc[ts]['cci']
#         print('cci = {}'.format(cci))
#         print('end')
        
        if (
#             kbars.iloc[ts - 3:ts]['rsi'].min() < 30 and
#             kbars.iloc[ts - 1]['macdhist'] < 0 and
            current_time < date.replace(hour=13, minute=20, second=0) and
            kbars.iloc[ts]['cci'] > -100 and
            kbars.iloc[ts]['macdhist'] > 0 and
            position == 0
        ):
            
            position = 1          
            entry_price = current_price
            entry_time = current_time.time()

            print('[{}] buy {} at {}'.format(current_time, code, current_price))
        
        elif (
#             kbars.iloc[ts - 3:ts]['rsi'].max() > 70 and
#             kbars.iloc[ts - 1]['macdhist'] > 0 and
            kbars.iloc[ts]['cci'] > 100 and
            kbars.iloc[ts]['macdhist'] > 0 and
            position != 0
        ):
            exit_price = current_price
            exit_time = current_time.time()
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
        elif (
            current_time >= date.replace(hour=13, minute=20, second=0) and
            position != 0
        ):
            
            exit_price = current_price
            exit_time = current_time.time()
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
    if entry_time and exit_time:
        transaction = pd.DataFrame([[date, 
                                     code, 
                                     entry_time,
                                     entry_price, 
                                     position * 1000,
                                     entry_price * position * 1000,
                                     exit_time,
                                     exit_price,
                                     position * 1000,
                                     exit_price * position * 1000]
                                   ],
                                    columns=[
                                        '成交日期', 
                                        '股票代號',
                                        '買進時間',
                                        '買進價格',
                                        '買進股數',
                                        '買進金額',
                                        '賣出時間',
                                        '賣出價格',
                                        '賣出股數',
                                        '賣出金額'])
        return transaction
    else:
        return pd.DataFrame()

In [6]:
day_trading_backtest('2456', pd.to_datetime('2021/3/2'), connection, api)

<SecurityType.Option: 'OPT'> fetch done.
 start current_time = 2021-03-02 09:05:00
current_price = 114.0
[2021-03-02 09:05:00] buy 2456 at 114.0
 start current_time = 2021-03-02 09:10:00
current_price = 112.5
[2021-03-02 09:10:00] sell 2456 at 112.5


,成交日期,股票代號,買進時間,買進價格,買進股數,買進金額,賣出時間,賣出價格,賣出股數,賣出金額
0,2021-03-02,2456,09:05:00,114.0,1000,114000.0,09:10:00,112.5,1000,112500.0
